In [1]:
# Thermostat UI2

In [2]:
import requests
from ipywidgets import Button, HBox, IntProgress, IntSlider, Label, Layout, VBox
import time
import logging
from utils import *

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    }

GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                     ?meas saref:hasValue ?temp .
                     ?meas saref:isMeasuredIn saref:TemperatureUnit .
                     ?meas saref:hasTimestamp ?timestamp .
                     ?meas saref:isMeasurementOf ?room_id .
                     ?meas saref:relatesToProperty saref:Temperature .
                     ?meas saref:measurementMadeBy ?device_id ."""

In [4]:
def present_measurement(binding: dict[str, str], historical: bool = False):
    if historical:
        print(
            f"[HISTORICAL] Temperature was {binding['temp']} units at {binding['timestamp'][1:-1]}",
            flush=True,
        )
    else:
        print(
            f"[NEW!] Live temperature is {binding['temp']} units at {binding['timestamp'][1:-1]}",
            flush=True,
        )


def handle_react_measurements(bindings):
    for binding in bindings:
        present_measurement(binding)
    return []

In [5]:
def start_ui_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)
    
    post_ki_id = register_post_knowledge_interaction(
        GRAPH_PATTERN,
        None,
        "post-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    react_measurements_ki = register_react_knowledge_interaction(
        GRAPH_PATTERN,
        None,
        "react-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    start_handle_loop(
        {
            react_measurements_ki: handle_react_measurements,
        },
        kb_id,
        ke_endpoint,
    )


In [6]:
start_ui_kb(
    "http://example.org/ui",
    "UI",
    "UI for measurement",
    "http://knowledge_engine:8280/rest/",
)

INFO:utils:registered UI
INFO:utils:received issued knowledge interaction id: http://example.org/ui/interaction/react-measurements


[NEW!] Live temperature is 10 units at 2023-06-18T20:28:35+00:00
[NEW!] Live temperature is 6 units at 2023-06-18T20:28:40+00:00
[NEW!] Live temperature is 6 units at 2023-06-18T20:28:46+00:00
[NEW!] Live temperature is 6 units at 2023-06-18T20:28:52+00:00
[NEW!] Live temperature is 8 units at 2023-06-18T20:28:58+00:00
[NEW!] Live temperature is 6 units at 2023-06-18T20:29:04+00:00
[NEW!] Live temperature is 10 units at 2023-06-18T20:29:10+00:00
[NEW!] Live temperature is 6 units at 2023-06-18T20:29:15+00:00
[NEW!] Live temperature is 8 units at 2023-06-18T20:29:21+00:00
[NEW!] Live temperature is 8 units at 2023-06-18T20:29:27+00:00
[NEW!] Live temperature is 10 units at 2023-06-18T20:29:33+00:00
[NEW!] Live temperature is 9 units at 2023-06-18T20:29:39+00:00
[NEW!] Live temperature is 6 units at 2023-06-18T20:29:45+00:00
[NEW!] Live temperature is 11 units at 2023-06-18T20:29:50+00:00
[NEW!] Live temperature is 5 units at 2023-06-18T20:29:56+00:00
[NEW!] Live temperature is 5 units a

KeyboardInterrupt: 

In [ ]:
thermostat = requests.get(THERMOSTAT_API_URL).json()

name_label = Label(f"Name: {thermostat['name']}")
mode_label = Label(f"Mode: {thermostat['mode']}")

desired_temp_label = Label(f"Desired Temperature: {thermostat['desired_temperature']}")
desired_temp_slider = IntSlider(
    value=thermostat["desired_temperature"], min=0, max=100, orientation="vertical"
)
desired_temp_progress = IntProgress(
    value=thermostat["actual_temperature"],
    min=0,
    max=100,
    orientation="vertical",
    layout=Layout(height="200px"),
)
desired_temp_box = VBox([HBox([desired_temp_slider]), desired_temp_label])

actual_temp_label = Label(f"Actual Temperature: {thermostat['actual_temperature']}")
actual_temp_progress = IntProgress(
    value=thermostat["actual_temperature"],
    min=0,
    max=100,
    orientation="vertical",
    layout=Layout(height="200px"),
)
actual_temp_box = VBox([actual_temp_progress, actual_temp_label])

submit_button = Button(description="Submit", button_style="danger")


In [ ]:

def update_desired_temp(change):
    thermostat["desired_temperature"] = change.new
    desired_temp_label.value = f"Desired Temperature: {change.new}"


def submit_changes(b):
    requests.put(
        THERMOSTAT_API_URL
        + "/desired_temperature"
        + f'?desired_temperature={thermostat["desired_temperature"]}'
    )
    update_state()


def update_state():
    thermostat.update(requests.get(THERMOSTAT_API_URL).json())
    name_label.value = f"Name: {thermostat['name']}"
    mode_label.value = f"Mode: {thermostat['mode']}"
    desired_temp_label.value = (
        f"Desired Temperature: {thermostat['desired_temperature']}"
    )
    
actual_temp_label.value = f"Actual Temperature: {thermostat['actual_temperature']}"
actual_temp_progress.value = thermostat["actual_temperature"]
desired_temp_slider.observe(update_desired_temp, names="value")
submit_button.on_click(submit_changes)


In [ ]:
display(
    VBox(
        [
            name_label,
            mode_label,
            HBox([desired_temp_box, actual_temp_box]),
            submit_button,
        ]
    )
)